In [ ]:
import jax.numpy as np
import numpy as onp
from jax import jit
from jax import random
from jax import lax

from jax.config import config
# config.update('jax_enable_x64', True)

from jax_md import space
from jax_md import energy
from jax_md import simulate
from jax_md import quantity
from jax_md import partition

: 

In [ ]:
4 +3

: 

In [ ]:
lattice_constant = 1.37820
N_rep = 40
box_size = N_rep * lattice_constant
# Using float32 for positions / velocities, but float64 for reductions.
dtype = np.float32

# Specify the format of the neighbor list. 
# Options are Dense, Sparse, or OrderedSparse. 
format = partition.OrderedSparse

displacement, shift = space.periodic(box_size)